# Fitting data with `easyCore`

The `easyCore` library is designed to enable the model-dependent analysis, using a purely Python interface and give access to a range of optimization algorithms. 
We can perform the analysis of any data for which we have a closed-form mathematical description, the parameters of which we want to refine. 

In this short demonstration, we will look at how `easyCore` can be used to analyse the toy problem of data that follows a quadratic relationship. 
We will manufacture some quadratic data to work with below. 
`easyCore` comes packages with an internal version of `numpy` that we will use throughout and set the random seed to ensure reproducibility in this example. 


In [ ]:
from easyCore import np

np.random.seed(123)

a_true = -0.9594
b_true = 7.294
c_true = 3.102

N = 50
x = np.sort(10 * np.random.rand(N))
yerr = 0.1 + 3 * np.random.rand(N)
y = a_true * x ** 2 + b_true * x + c_true
y += np.abs(y) * 0.2 * np.random.randn(N)

With our data created, lets have a look at it.

In [ ]:
import matplotlib.pyplot as plt

plt.errorbar(x, y, yerr, marker='.', ls='', color='k')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

The data looks reasonable, so we can start to construct the `easyCore` analysis model. 

This will consist of three parameters, which we will call `a`, `b` and `c`. 
We will give these initial values that are the same as the true values defined above and cause we want to optimize these parameters, we will set them not to be fixed. 

In [ ]:
from easyCore.Objects.Variable import Parameter

a = Parameter(name='a', value=a_true, fixed=False)
b = Parameter(name='b', value=b_true, fixed=False)
c = Parameter(name='c', value=c_true, fixed=False)

The mathematical model that we are looking to optimize is

```{math}
:label: quadratic
y = a x ^ 2 + b x + c.
```

We can create a function that implements this mathematical model as shown below. 
Note, that it is necessary to include the `*args`, and `**kwargs` arguments and to use the `raw_value` property of each parameter. 

In [ ]:
def math_model(x, *args, **kwargs):
    return a.raw_value * x ** 2 + b.raw_value * x + c.raw_value

We can now plot the initial guess of this mathematical model along with the experimental data.  

In [ ]:
plt.errorbar(x, y, yerr, marker='.', ls='', color='k')
plt.plot(x, math_model(x), '-')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.show()

Now we can begin the process of using `easyCore` to optimize the parameters `a`, `b`, and `c` and therefore fit the model to the data.
First, we create a `BaseObj` that collects together the parameters to be optimized and then, using this and our mathematical model, create a `Fitter`. 

In [ ]:
from easyCore.Objects.ObjectClasses import BaseObj
from easyCore.Fitting.Fitting import Fitter

quad = BaseObj(name='quad', a=a, b=b, c=c)
f = Fitter(quad, math_model)

We can then use `easyCore` to obtain the {term}`maximum likelihood estimation` (MLE) parameters for the quadratic model.
The `y` describes the position of the normal distributions for the data while the `weights` the reciprocal of their width.

In [ ]:
res = f.fit(x=x, y=y, weights=1/yerr)

With the MLE found, we can print the paramaters out to see the optimized values and estimated statistical uncertainties. 

In [ ]:
a, b, c

Additionally, we can thne plot the optimized model with the data. 

In [ ]:
plt.errorbar(x, y, yerr, marker='.', ls='', color='k')
plt.plot(x, math_model(x), '-')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.show()

This approach to use `easyCore` for the optimization of mathematical models can be applied to many different use cases, including in neutron scattering. 
In the exercises, you will look to analyse your simulated and reduced data using `easyCore`. 